# ID BLOCK
- @author Eric Zair
- @zairea200@potsdam.edu

# Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using Turi Create. Now we will use Turi Create along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a Turi Create SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Fire up Turi Create

Make sure you have the latest version of Turi Create

In [125]:
from turicreate import SFrame

# Load in house sales data

load data from home_dats.sframe

In [126]:
sales = SFrame("../Multiple_input/home_data.sframe")

# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use Turi Create's built in .to_dataframe() which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's .as_matrix() to convert the dataframe into a numpy matrix.

import numpy here

In [127]:
import numpy as np

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

In [128]:
features = sales.column_names()
features.remove('price')


def get_numpy_data(data_sframe, features, output):
    data_sframe.to_numpy()

    # This simply adds the constant column that we will need.
    data_sframe['constant'] = 1
    features = ['constant'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()

    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    print(output_array.shape)
    return feature_matrix, output_array

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [129]:
# the [] around 'sqft_living' makes it a list
example_features, example_output = get_numpy_data(sales, ['sqft_living'], 'price') 
# this accesses the first row of the data the ':' indicates 'all columns'
print(example_features[0,:])

print(example_output[0]) 

(21613,)
[1.00e+00 1.18e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrays we can use np.dot() to compute this:

In [130]:
my_weights = np.array([0., 0.])
#  use the first data point of example_features and calculate the dot product 
#  and that should be 1181.0
# yi_hat is our predicted. value.
print(my_weights.shape)
my_features  = example_features[0,]
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

(2,)
0.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [131]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as 
    # columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions =  np.dot(feature_matrix, weights)
    return predictions

If you want to test your code run the following cell:

In [132]:
test_predictions = predict_output(example_features, my_weights)

# should be 1181 example_feature
print(test_predictions[0])

# should be 2571.0
print(test_predictions[1])

0.0
0.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives, we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [133]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same 
    # length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' 
    # and return the value.
    # g decent is just partial derivaive of each features (all of them).
    print(errors)
    derivative =  -2 * np.dot(errors, feature)

    print("errors", errors.shape)

    print("features:", feature.shape)
    return derivative

Test your feature derivartive for single feature Sqft_living and price as output.
Your weights for testing are 0's.

In [134]:
example_features, example_output = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 

# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output

# the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 
#'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative and it is -23345850022.0

(21613,)
[-221900. -538000. -180000. ... -402101. -400000. -325000.]
errors (21613,)
features: (21613,)
23345850022.0
-23345850022.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [135]:
from math import sqrt

In [136]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)

    while not converged:
        # Calucate our prediction.
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output
        gradient_sum_of_squares = 0

        # Do partial derivative for each thing.
        for i in range(len(weights)):
            derivative = feature_derivative(errors, feature_matrix[:,i])
            derivative_squared = derivative ** 2
            gradient_sum_of_squares += derivative_squared
            weights[i] += (step_size * derivative)
        
        # Now we need to update magnitude, to know if we need to converge.
        gradient_magnitude = sqrt(gradient_sum_of_squares)
        if gradient_magnitude < tolerance:
            converged = True

    return weights

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature, the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [137]:
train_data, test_data = sales.random_split(.8,seed=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimate the parameters in the simple regression on squarefeet. The following cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [138]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
simple_feature_matrix, output = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7
print(initial_weights)

(17384,)
[-4.7e+04  1.0e+00]


Next run your gradient descent with the above parameters.

In [139]:
test_weights = regression_gradient_descent(simple_feature_matrix,
                                           output, initial_weights,
                                           step_size, tolerance)
print(test_weights)

[-267720. -582430. -226230. ... -405470. -445400. -370980.]
errors (17384,)
features: (17384,)
[-267720. -582430. -226230. ... -405470. -445400. -370980.]
errors (17384,)
features: (17384,)
[149835.75268253 326992.10740189  46242.87107465 ... 135936.99307949
 120777.24115888 -10041.95721323]
errors (17384,)
features: (17384,)
[149835.75268253 326992.10740189  46242.87107465 ... 135936.99307949
 120777.24115888 -10041.95721323]
errors (17384,)
features: (17384,)
[  41402.97905279   90829.24562979  -24514.12116057 ...   -4658.0329602
  -26250.23536279 -103771.98688413]
errors (17384,)
features: (17384,)
[  41402.97905279   90829.24562979  -24514.12116057 ...   -4658.0329602
  -26250.23536279 -103771.98688413]
errors (17384,)
features: (17384,)
[ 69561.29400806 152157.08979222  -6139.62417287 ...  31852.3217235
  11930.52726658 -79431.74723328]
errors (17384,)
features: (17384,)
[ 69561.29400806 152157.08979222  -6139.62417287 ...  31852.3217235
  11930.52726658 -79431.74723328]
errors (1

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

In [140]:
import turicreate
model_for_sqft = turicreate.linear_regression.create(train_data, target=my_output, features=simple_features, validation_set=None)
print(model_for_sqft.coefficients)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 1

Number of unpacked features : 1

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.003757     | 4349521.926170     | 262943.613754                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None | -47114.0206702149 | 4923.344377526256 |
| sqft_living |  None | 281.9578501659871 | 2.164054653233476 |
+-------------+-------+-------------------+-------------------+
[2 rows x 4 columns]



Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [141]:
test_features, test_output = get_numpy_data(test_data, simple_features, my_output)
print(test_features)
print(test_output)

(4229,)
[[1.00e+00 1.43e+03]
 [1.00e+00 2.95e+03]
 [1.00e+00 1.71e+03]
 ...
 [1.00e+00 2.52e+03]
 [1.00e+00 2.31e+03]
 [1.00e+00 1.02e+03]]
[310000. 650000. 233000. ... 610685. 400000. 402101.]


Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [142]:
predictions = predict_output(test_features, test_weights)
print(predictions)

[356134.44317093 784640.86422788 435069.83652353 ... 663418.65300782
 604217.10799338 240550.4743332 ]


**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [143]:
print(predictions[0].round())

356134.0


Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [144]:
rss_errors = predictions - test_output
# Then square and add them up

RSS = (rss_errors**2).sum()
print(RSS)

275400047593155.94


# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [145]:
model_features = ['sqft_living', 'sqft_living15'] 
# sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights_2 = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9
print (feature_matrix)

(17384,)
[[1.00e+00 1.18e+03 1.34e+03]
 [1.00e+00 2.57e+03 1.69e+03]
 [1.00e+00 7.70e+02 2.72e+03]
 ...
 [1.00e+00 1.53e+03 1.53e+03]
 [1.00e+00 1.60e+03 1.41e+03]
 [1.00e+00 1.02e+03 1.02e+03]]


Use the above parameters to estimate the model weights for train data. Record these values for your quiz.

In [146]:
reg_weights = regression_gradient_descent(feature_matrix, output, initial_weights_2, step_size, tolerance)
print('intercept: {}'.format(reg_weights[0]))
print('sqft: {}'.format(reg_weights[1]))
print('sqft 15: {}'.format(reg_weights[2]))

2 -108420.42181793]
errors (17384,)
features: (17384,)
[  54822.00408109  101994.10309769   86756.29890389 ...   14868.27972028
  -15855.134504   -108421.13637627]
errors (17384,)
features: (17384,)
[  54822.00408109  101994.10309769   86756.29890389 ...   14868.27972028
  -15855.134504   -108421.13637627]
errors (17384,)
features: (17384,)
[  54822.00408109  101994.10309769   86756.29890389 ...   14868.27972028
  -15855.134504   -108421.13637627]
errors (17384,)
features: (17384,)
[  54819.81347433  101999.95508273   86738.90995621 ...   14867.23331717
  -15854.58578587 -108421.83398083]
errors (17384,)
features: (17384,)
[  54819.81347433  101999.95508273   86738.90995621 ...   14867.23331717
  -15854.58578587 -108421.83398083]
errors (17384,)
features: (17384,)
[  54819.81347433  101999.95508273   86738.90995621 ...   14867.23331717
  -15854.58578587 -108421.83398083]
errors (17384,)
features: (17384,)
[  54817.67484252  102005.66822181   86721.93358387 ...   14866.2117413
  -15854.

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [147]:
multiple_features, multiple_output = get_numpy_data(test_data, model_features, my_output)
multiple_predictions = predict_output(multiple_features, reg_weights)
print(multiple_predictions)

(4229,)
[366651.41203656 762662.39786164 386312.09499712 ... 682087.39928241
 585579.27865729 216559.20396617]


**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [148]:
print(multiple_predictions[0].round())

366651.0


What is the actual price for the 1st house in the test data set?

In [149]:
print(test_data[0]['price'])

310000.0


**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

### It appears that the first model was closer, but still is by thousands. Each model comepletly overestimates.

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [150]:
err = multiple_predictions - multiple_output
RSS_model_2 = (err**2).sum()
print(RSS_model_2)

270263446465244.06


**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **

In [151]:
print("First Model: {}".format(RSS))
print("Second Model: {}".format(RSS_model_2))

First Model: 275400047593155.94
Second Model: 270263446465244.06


### Model to is lower.